In [1]:
import numpy as np
import cv2
import time
cap=cv2.VideoCapture(0)#VideoCapture:Class for video capturing from video files, image sequences or cameras.
                       #cv2.VideoCapture(device):id of the opened video capturing device (i.e. a camera index). 
                       #If there is a single camera connected, just pass 0.
fourcc=cv2.VideoWriter_fourcc(*'XVID') 
out=cv2.VideoWriter('harry.avi' , fourcc, 20.0, (640,480))
#FourCC is a 4-byte code used to specify the video codec. The codecs for Windows is DIVX. Therefore *’XVID' is used.
#Then to save Operated Video from a webcam, cv2.VideoWriter() function is used.

time.sleep(2) #to halt the execution of the program for given time in seconds.

background=0 #capturing background
for i in range(30):
    ret,background=cap.read() #capturing image

while(cap.isOpened()): #Returns true if video capturing has been initialized already.
    ret, img = cap.read() #.read():Grabs, decodes and returns the next video frame.
    if not ret:
        break
        
    hsv=cv2.cvtColor(img, cv2.COLOR_BGR2HSV) #.cvtColor():Converts BGR image into HSV image.
    lower_red = np.array([0,120,70])
    upper_red = np.array([10,255,255])
    mask1 = cv2.inRange(hsv , lower_red , upper_red) 
    
    lower_red = np.array([170,120,70])
    upper_red = np.array([180,255,255])
    mask2 = cv2.inRange(hsv , lower_red , upper_red)
    
    #in above steps, we find the pixel location of red color and store them in mask1 and mask2. The HSV range of true red is 
    #(0,255,255) and to accomodate variations of it we will consider the above ranges.
    #To learn more check:- http://hanzratech.in/2015/02/07/caveat-thresholding-hue-component.html
    
    mask1 = mask1 + mask2 #OR
    
    #in order to carry out process of Dilution and Erosion:-
    mask1=cv2.morphologyEx(mask1, cv2.MORPH_OPEN ,np.ones((3,3) , np.uint8) , iterations=2) #to remove small objects.
        
    mask2=cv2.morphologyEx(mask1, cv2.MORPH_DILATE ,np.ones((3,3) , np.uint8) , iterations=1) #to make image wider.
        
    mask2 = cv2.bitwise_not(mask1) #to invert every bit of an array.
    
    res1 = cv2.bitwise_and(background, background, mask=mask1) #calculates the per-element bit-wise conjunction of two 
    res2 = cv2.bitwise_and(img, img, mask=mask2)               #arrays or an array and a scalar.
    
    final_output = cv2.addWeighted(res1 , 1, res2 , 1, 0) #calculates the weighted sum of two arrays.
                   #cv2.addWeighted(array,alpha,array,beta,gamma):- alpha,beta(weights of arrays) gamma(scalar added to each sum)
    
    cv2.imshow('Harry' , final_output)
    
    if cv2.waitKey(10) & 0xFF==ord('q'): #to cleanup the camera and close any open windows
        cap.release()
        cv2.destroyAllWindows()
        break